In [1]:
import pickle
import tensorflow
from keras import Model
from keras.layers import (
    Input,
    TimeDistributed,
    LSTM,
    Dense,
    Input,
    Dense,
    Conv1D,
    Flatten, 
    Embedding,
    Dropout)
from imblearn.over_sampling import SMOTE
import numpy
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [2]:
%run constants.py


In [3]:
with open(X_TRAIN_INPUT_SAVE_FILE, "rb") as f:
        x_train = pickle.load(f)
with open(Y_TRAIN_INPUT_SAVE_FILE, "rb") as f:
        y_train = pickle.load(f)
with open(X_TEST_INPUT_SAVE_FILE, "rb") as f:
        x_test = pickle.load(f)
with open(Y_TEST_INPUT_SAVE_FILE, "rb") as f:
        y_test = pickle.load(f)
with open(X_VAL_INPUT_SAVE_FILE, "rb") as f:
        x_val = pickle.load(f)
with open(Y_VAL_INPUT_SAVE_FILE, "rb") as f:
        y_val = pickle.load(f)
with open(EMBEDDING_MATRIX_SAVE_FILE, "rb") as f:
        embedding_matrix = pickle.load(f)

print("------Saving varaibles for reuse ------")
print(f"X_train shape: {x_train.shape}")
print(f"Y_train shape: {y_train.shape}")
print(f"X_test shape: {x_test.shape}")
print(f"Y_test shape: {y_test.shape}")
print(f"X_val shape: {x_val.shape}")
print(f"Y_val shape: {y_val.shape}")
print(f"Embedding shape: {embedding_matrix.shape}")
print(f"Total 0 values: {(y_train == 0).sum()}")
print(f"Total 1 values: {(y_train == 1).sum()}")

------Saving varaibles for reuse ------
X_train shape: (2465, 30, 350)
Y_train shape: (2465,)
X_test shape: (685, 30, 350)
Y_test shape: (685,)
X_val shape: (274, 30, 350)
Y_val shape: (274,)
Embedding shape: (10000, 300)
Total 0 values: 1346
Total 1 values: 1119


In [4]:
if BALANCE_DATA:
    arr = x_train.reshape(len(x_train), -1)
    print(arr.shape)
    sm = SMOTE(random_state=SEED)
    x_train_bal, y_train = sm.fit_resample(arr, y_train.ravel())
    print(f"After OverSampling, the shape of train_X: {x_train_bal.shape}")
    print(f"After OverSampling, the shape of train_y: {y_train.shape}")
    print(f"After OverSampling, counts of label '1': {sum(y_train == 1)}")
    print(f"After OverSampling, counts of label '0': {sum(y_train == 0)}")
    x_train = numpy.reshape(x_train_bal, (-1, x_train.shape[1], x_train.shape[2]))
x_train.shape


(2465, 10500)
After OverSampling, the shape of train_X: (2692, 10500)
After OverSampling, the shape of train_y: (2692,)
After OverSampling, counts of label '1': 1346
After OverSampling, counts of label '0': 1346


(2692, 30, 350)

In [5]:
embedding_layer = Embedding(
        MAX_VOCAB_SIZE,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,)


In [6]:
def run_stacked_lstm():
    document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    embedding_sequences =embedding_layer(document_input)
    x = LSTM(32, return_sequences=True)(embedding_sequences)
    x = LSTM(32)(x)
    x = Dropout(0.3)(x)
    doc_model = Model(document_input, x)
    input_docs = Input(
                shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
            )

    x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
    x = LSTM(32)(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(input_docs, outputs)

    opt = tensorflow.keras.optimizers.Adam(learning_rate=LR, beta_1=0.5, beta_2=0.999)

    model.compile(
                loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"]
            )
    model.summary()
    model.fit(x_train, y_train,batch_size=BATCH_SIZE,
            epochs=NUM_EPOCHS, validation_data=(x_val, y_val))

In [7]:
def run_lstm():
    document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    embedding_sequences =embedding_layer(document_input)
    x = LSTM(32)(embedding_sequences)
    x = Dropout(0.3)(x)
    doc_model = Model(document_input, x)
    input_docs = Input(
                shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
            )

    x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
    x = LSTM(32)(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(input_docs, outputs)

    opt = tensorflow.keras.optimizers.Adam(learning_rate=LR, beta_1=0.5, beta_2=0.999)

    model.compile(
                loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"]
            )
    model.summary()
    model.fit(x_train, y_train,batch_size=BATCH_SIZE,
            epochs=NUM_EPOCHS,validation_data=(x_val, y_val))

In [8]:
def run_cnn():
    document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    embedding_sequences = embedding_layer(document_input)

    x = Conv1D(filters=300, kernel_size=5, padding="valid")(embedding_sequences)
    x = Dropout(0.3)(x)
    doc_model = Model(document_input, x)

    input_docs = Input(
            shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
        )

    x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
    x = Conv1D(filters=300, kernel_size=5, padding="valid")(x)
    x = Dropout(0.3)(x)
    x = Flatten()(x)
    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(input_docs, outputs)

    opt = tensorflow.keras.optimizers.Adam(learning_rate=LR, beta_1=0.5, beta_2=0.999)
    model.compile(
                loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"]
            )
    model.summary()
    model.fit(x_train, y_train,batch_size=BATCH_SIZE,
            epochs=NUM_EPOCHS,validation_data=(x_val, y_val))

In [9]:

run_stacked_lstm()
run_lstm()
run_cnn()

2022-07-24 03:01:40.456181: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-24 03:01:41.319489: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 566 MB memory:  -> device: 0, name: GeForce RTX 2070, pci bus id: 0000:03:00.0, compute capability: 7.5
2022-07-24 03:01:41.320324: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 6414 MB memory:  -> device: 1, name: GeForce RTX 2070, pci bus id: 0000:04:00.0, compute capability: 7.5


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_docs (InputLayer)     [(None, 30, 350)]         0         
                                                                 
 token_embedding_model (Time  (None, 30, 32)           3050944   
 Distributed)                                                    
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 3,059,297
Trainable params: 59,297
Non-trainable params: 3,000,000
____________________________________________

2022-07-24 03:01:56.740025: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 192.26MiB (rounded to 201600000)requested by op model_1/token_embedding_model/embedding/embedding_lookup
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-07-24 03:01:56.740093: I tensorflow/core/common_runtime/bfc_allocator.cc:1027] BFCAllocator dump for GPU_0_bfc
2022-07-24 03:01:56.740117: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (256): 	Total Chunks: 46, Chunks in use: 45. 11.5KiB allocated for chunks. 11.2KiB in use in bin. 644B client-requested in use in bin.
2022-07-24 03:01:56.740134: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (512): 	Total Chunks: 10, Chunks in use: 10. 5.0KiB allocated for chunks. 5.0KiB in use in bin. 5.0KiB client-requested 

ResourceExhaustedError: Graph execution error:

Detected at node 'model_1/token_embedding_model/embedding/embedding_lookup' defined at (most recent call last):
    File "/home/aaron/anaconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/aaron/anaconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/aaron/anaconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/aaron/anaconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/home/aaron/anaconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/aaron/anaconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.ru